# Tensorflow Backdoor Poisoning MNIST Demo 

This demo will cover a basic backdoor poisoning attack on an MNIST-LeNet model.
The following two sections cover experiment setup and is similar across all demos.
To access demo results in MlFlow, please follow the general experiment setup steps outlined in `basic-mlflow-demo`.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the MNIST dataset directory. Currently set to `/nfs/data/Mnist` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Mnist"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_backdoor_poison"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Path to custom task plugins archives
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz")

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [2]:
%%bash

# Create the workflows.tar.gz file
make workflows

tar  -czf workflows.tar.gz src/gaussian_augmentation.py src/infer.py src/train_with_madry_pgd_defense.py src/tasks.py src/deploy_poison_data.py src/gen_poison_data.py src/train.py src/import_keras.py src/spatial_smoothing.py src/jpeg_compression.py src/init_model.py src/gen_poison_clean_data.py src/gen_poison_model.py MLproject
chmod 644 workflows.tar.gz


/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [3]:
restapi_client = utils.SecuringAIClient()

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [4]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 24,
 'createdOn': '2021-02-11T09:13:02.448572',
 'lastModified': '2021-02-11T09:13:02.448572',
 'name': 'howard_mnist_backdoor_poison'}

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [5]:
restapi_client.list_queues()

[{'name': 'tensorflow_cpu',
  'createdOn': '2020-11-20T17:46:06.756687',
  'lastModified': '2020-11-20T17:46:06.756687',
  'queueId': 1},
 {'name': 'tensorflow_gpu',
  'createdOn': '2020-11-20T18:00:40.876888',
  'lastModified': '2020-11-20T18:00:40.876888',
  'queueId': 2},
 {'name': 'pytorch_cpu',
  'createdOn': '2020-11-20T19:52:36.079781',
  'lastModified': '2020-11-20T19:52:36.079781',
  'queueId': 5},
 {'name': 'pytorch_gpu',
  'createdOn': '2020-11-20T19:52:43.348460',
  'lastModified': '2020-11-20T19:52:43.348460',
  'queueId': 7}]

This example also makes use of the `custom_fgm_patch_poisoning_plugins` custom task plugin package stored locally under the `task-plugins/securingai_custom/custom_fgm_patch_poisoning_plugins` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [6]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

tar -C task-plugins/securingai_custom -czf custom-plugins-evaluation.tar.gz custom_fgm_patch_poisoning_plugins/defenses_image_preprocessing.py custom_fgm_patch_poisoning_plugins/attacks_poison.py custom_fgm_patch_poisoning_plugins/tracking_mlflow_updated.py custom_fgm_patch_poisoning_plugins/tensorflow.py custom_fgm_patch_poisoning_plugins/import_keras.py custom_fgm_patch_poisoning_plugins/defenses_training.py custom_fgm_patch_poisoning_plugins/__init__.py custom_fgm_patch_poisoning_plugins/registry_art.py custom_fgm_patch_poisoning_plugins/estimators_keras_classifiers.py custom_fgm_patch_poisoning_plugins/data_tensorflow.py
chmod 644 custom-plugins-evaluation.tar.gz


Now that the custom task plugin package is packaged into an archive file, next we register it by uploading the file to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [7]:
restapi_client.delete_custom_task_plugin(name="custom_fgm_patch_poisoning_plugins")
response_custom_plugins = restapi_client.get_custom_task_plugin(name="custom_fgm_patch_poisoning_plugins")

if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="custom_fgm_patch_poisoning_plugins",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

response_custom_plugins

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'modules': ['data_tensorflow.py',
  'tensorflow.py',
  'tracking_mlflow_updated.py',
  'registry_art.py',
  'defenses_training.py',
  'attacks_poison.py',
  '__init__.py',
  'estimators_keras_classifiers.py',
  'import_keras.py',
  'defenses_image_preprocessing.py'],
 'taskPluginName': 'custom_fgm_patch_poisoning_plugins',
 'collection': 'securingai_custom'}

If at any point you need to update one or more files within the `evaluation` plugin package, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="custom_fgm_patch_poisoning_plugins")
```

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [8]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response


# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

## MNIST Training: Baseline Model

Next, we need to train our baseline model that will serve as a reference point for the effectiveness of our attacks.
We will be submitting our job to the `"tensorflow_gpu"` queue.

In [9]:
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

response_le_net_train = get_run_id(response_le_net_train)
print_mlflow_results(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-07-15T06:51:17.324628',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_backdoor_poison_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 24,
 'jobId': '53c2cf79-5dcb-46d5-8fa1-e06a22f993a6',
 'lastModified': '2021-07-15T06:51:17.324628',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/01705f698773451fbc4c0c8f037eb61a/workflows.tar.gz'}
{'accuracy': 0.9770227670669556,
 'auc': 0.9988901019096375,
 'loss': 0.0778634323800676,
 'precision': 0.9807555079460144,
 'recall': 0.9735230207443237,
 'restored_epoch': 2.0,
 'stopped_epoch': 7.0,
 'training_time_in_minutes': 5.387063266666667,
 'val

### Generating Poisoned Images

Now we will create our set of poisoned images.
Start by submitting the poison generation job below.

In [10]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


In [11]:
## Create poisoned test images.
response_gen_poison_le_net_test = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_data",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=1",
            "-P label_type=test"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_test)
print("")

response_gen_poison_le_net_test = get_run_id(response_gen_poison_le_net_test)

Backdoor poison attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-07-15T06:57:06.598260',
 'dependsOn': '53c2cf79-5dcb-46d5-8fa1-e06a22f993a6',
 'entryPoint': 'gen_poison_data',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/testing -P batch_size=100 -P '
                     'target_class=1 -P poison_fraction=1 -P label_type=test',
 'experimentId': 24,
 'jobId': 'ff122ba8-728e-4a4d-b077-342334101e1b',
 'lastModified': '2021-07-15T06:57:06.598260',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/8156fb630fe64977bbab94992d6963c0/workflows.tar.gz'}



/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
## Create poisoned training images.
response_gen_poison_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_data",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/training",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=0.1",
            "-P label_type=train"
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_train)
print("")

response_gen_poison_le_net_train = get_run_id(response_gen_poison_le_net_train)

Backdoor poison attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-07-15T06:57:13.745067',
 'dependsOn': '53c2cf79-5dcb-46d5-8fa1-e06a22f993a6',
 'entryPoint': 'gen_poison_data',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/training -P batch_size=100 '
                     '-P target_class=1 -P poison_fraction=0.1 -P '
                     'label_type=train',
 'experimentId': 24,
 'jobId': 'a7d74f7f-2b6a-44fc-a92b-5f7059f2c6ef',
 'lastModified': '2021-07-15T06:57:13.745067',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/c6587710f2284204806388c1914e70b5/workflows.tar.gz'}



In [13]:
# Train new model on poisoned training images
response_le_net_poison_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_data_poison_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
        "-P load_dataset_from_mlruns=true",
        f"-P dataset_run_id_training={response_gen_poison_le_net_train['mlflowRunId']}",
        "-P adv_tar_name=adversarial_poison.tar.gz",
        "-P adv_data_dir=adv_poison_data"
    ]),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_train ["jobId"],
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_poison_model)

response_le_net_poison_model = get_run_id(response_le_net_poison_model)    

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-07-15T06:57:20.896888',
 'dependsOn': 'a7d74f7f-2b6a-44fc-a92b-5f7059f2c6ef',
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_backdoor_poison_data_poison_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing -P '
                     'load_dataset_from_mlruns=true -P '
                     'dataset_run_id_training=599f6c80f57f48e28e2bbc82138a21c9 '
                     '-P adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 24,
 'jobId': 'c15601a5-3965-4c39-a10a-4a759994e26b',
 'lastModified': '2021-07-15T06:57:20.896888',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflo

In [14]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [            
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_train["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)

print_mlflow_results(response_infer_reg_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-07-15T06:59:03.401582',
 'dependsOn': 'a7d74f7f-2b6a-44fc-a92b-5f7059f2c6ef',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=42575f76522241b9a25c69f044aff12c -P '
                     'model_name=howard_mnist_backdoor_poison_le_net -P '
                     'model_version=none -P batch_size=512 -P '
                     'adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 24,
 'jobId': '09c90906-b29f-4f7f-b1e8-8be25750e5a5',
 'lastModified': '2021-07-15T06:59:03.401582',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/6ab1441c32f549f0bc575404105054ac/workflows.tar.gz'}
{'accuracy': 0.9871000051498413,
 'auc': 0.9994533061981201,
 'loss': 0.03823211146518588,
 'precision': 0.9886603355407715,
 'recall': 0.9851999878883362}


In [15]:
# Inference: Poisoned model on poisoned test images.
response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [            
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_poison_model["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)

print_mlflow_results(response_infer_pos_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-07-15T06:59:25.884461',
 'dependsOn': 'c15601a5-3965-4c39-a10a-4a759994e26b',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=42575f76522241b9a25c69f044aff12c -P '
                     'model_name=howard_mnist_backdoor_poison_data_poison_le_net '
                     '-P model_version=none -P batch_size=512 -P '
                     'adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 24,
 'jobId': 'c046a0df-0a9d-4f8c-b10f-a82a0983fa40',
 'lastModified': '2021-07-15T06:59:25.884461',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/e5ff45268bce48b8a0a1a227af6bbf50/workflows.tar.gz'}
{'accuracy': 0.11349999904632568,
 'auc': 0.5077235698699951,
 'loss': 17.12834053039551,
 'precision': 0.11349999904632568,
 'recall': 0.11349999904632568}


## Defending against the adversarial backdoor poisoning attack

Now we will explore available defenses on the adversarial backdoor poisoning attack.
The following three jobs will run a selected defense (spatial smoothing, gaussian augmentation, or jpeg compression) and evaluate the defense on the baseline and backdoor trained models.

- The first job uses the selected defense entrypoint to apply a preprocessing defense over the poisoned test images.
- The second job runs the defended images against the poisoned backdoor model.
- The final job runs the defended images against the baseline model.

Ideally the defense will not impact the baseline model accuracy, while improving the backdoor model accuracy scores.

In [16]:
defenses = ["gaussian_augmentation", "spatial_smoothing", "jpeg_compression"]
defense = defenses[1]

response_poison_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point=defense,
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_poison.tar.gz",
            "-P dataset_name=adv_poison_data",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_test["jobId"],
)


print(f"{defense} defense (LeNet architecture) job submitted")
print("")
pprint.pprint(response_poison_def)
print("")

response_poison_def = get_run_id(response_poison_def)

spatial_smoothing defense (LeNet architecture) job submitted

{'createdOn': '2021-07-15T07:01:49.926120',
 'dependsOn': 'ff122ba8-728e-4a4d-b077-342334101e1b',
 'entryPoint': 'spatial_smoothing',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/testing -P batch_size=20 -P '
                     'load_dataset_from_mlruns=true -P '
                     'dataset_run_id=42575f76522241b9a25c69f044aff12c -P '
                     'dataset_tar_name=adversarial_poison.tar.gz -P '
                     'dataset_name=adv_poison_data',
 'experimentId': 24,
 'jobId': '7f478581-907b-4939-9120-cde1c3549945',
 'lastModified': '2021-07-15T07:01:49.926120',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/64d5792ca48b47d2baada33e7ff3fdc4/workflows.tar.gz'}



In [17]:
# Inference: Poisoned model on poisoned test images.
response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)
print_mlflow_results(response_infer_pos_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-07-15T07:01:57.090425',
 'dependsOn': '7f478581-907b-4939-9120-cde1c3549945',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=e29165c09ae54e1f846d7d0982afc32e -P '
                     'model_name=howard_mnist_backdoor_poison_data_poison_le_net '
                     '-P model_version=none -P batch_size=512 -P '
                     'adv_tar_name=spatial_smoothing_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 24,
 'jobId': '3895e56f-77e6-48e9-add8-71691fc98e73',
 'lastModified': '2021-07-15T07:01:57.090425',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/9e25b680e0b44b7184f3add611cb413f/workflows.tar.gz'}
{'accuracy': 0.9815999865531921,
 'auc': 0.999536395072937,
 'loss': 0.05960434582084417,
 'precision': 0.9850055575370789,
 'recall': 0.9787999987602234}


In [18]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)
print_mlflow_results(response_infer_reg_model)


Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-07-15T07:02:41.806242',
 'dependsOn': '7f478581-907b-4939-9120-cde1c3549945',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=e29165c09ae54e1f846d7d0982afc32e -P '
                     'model_name=howard_mnist_backdoor_poison_le_net -P '
                     'model_version=none -P batch_size=512 -P '
                     'adv_tar_name=spatial_smoothing_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 24,
 'jobId': 'bce976e9-7d1a-458c-a6bd-b420e83565a1',
 'lastModified': '2021-07-15T07:02:41.806242',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/a67dc72654d54e51b9b939e1b316d610/workflows.tar.gz'}
{'accuracy': 0.982200026512146,
 'auc': 0.9995614886283875,
 'loss': 0.05481360368430614,
 'precision': 0.9854109883308411,
 'recall': 0.9793999791145325}


<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [19]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [20]:
run_le_net = mlflow_client.get_run(response_le_net_train["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [21]:
pprint.pprint(run_le_net.data.metrics)

{'accuracy': 0.9770227670669556,
 'auc': 0.9988901019096375,
 'loss': 0.0778634323800676,
 'precision': 0.9807555079460144,
 'recall': 0.9735230207443237,
 'restored_epoch': 2.0,
 'stopped_epoch': 7.0,
 'training_time_in_minutes': 5.387063266666667,
 'val_accuracy': 0.9859119653701782,
 'val_auc': 0.999152660369873,
 'val_loss': 0.0466019456730561,
 'val_precision': 0.9876987338066101,
 'val_recall': 0.9839112758636475}


To review the run's parameters, we use,

In [22]:
pprint.pprint(run_le_net.data.params)

{'adv_data_dir': 'adv_patch_dataset',
 'adv_tar_name': 'adversarial_patch_dataset.tar.gz',
 'baseline': 'None',
 'batch_size': '256',
 'data_dir_testing': '/nfs/data/Mnist/testing',
 'data_dir_training': '/nfs/data/Mnist/training',
 'dataset_run_id_testing': "''",
 'dataset_run_id_training': "''",
 'dataset_seed': '2132508671',
 'entry_point_seed': '22674781371105050124662270946325565622',
 'epochs': '30',
 'image_size': '28,28,1',
 'imagenet_preprocessing': 'false',
 'learning_rate': '0.001',
 'load_dataset_from_mlruns': 'False',
 'min_delta': '-0.01',
 'model_architecture': 'le_net',
 'monitor': 'val_loss',
 'opt_amsgrad': 'False',
 'opt_beta_1': '0.9',
 'opt_beta_2': '0.999',
 'opt_decay': '0.0',
 'opt_epsilon': '1e-07',
 'opt_learning_rate': '0.001',
 'opt_name': 'Adam',
 'optimizer': 'Adam',
 'patience': '5',
 'register_model_name': 'howard_mnist_backdoor_poison_le_net',
 'restore_best_weights': 'True',
 'seed': '-1',
 'tensorflow_global_seed': '452972086',
 'validation_split': '0

To review the run's tags, we use,

In [23]:
pprint.pprint(run_le_net.data.tags)

{'mlflow.log-model.history': '[{"run_id": "84566053ed52470498a16c7154886702", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2021-07-15 06:56:56.603735", "flavors": '
                             '{"keras": {"keras_module": '
                             '"tensorflow_core.keras", "keras_version": '
                             '"2.2.4-tf", "data": "data"}, "python_function": '
                             '{"loader_module": "mlflow.keras", '
                             '"python_version": "3.7.10", "data": "data", '
                             '"env": "conda.yaml"}}}]',
 'mlflow.project.backend': 'securingai',
 'mlflow.project.entryPoint': 'train',
 'mlflow.source.name': '/work/tmpcel4yx7x',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'securingai',
 'securingai.dependsOn': 'None',
 'securingai.jobId': '53c2cf79-5dcb-46d5-8fa1-e06a22f993a6',
 'securingai.queue': 'tensorflow_gpu'}


There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).